
### Weather Tool Information Retrival

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
sys.path.append('../scripts')


In [ ]:
# Configuración de estilo
plt.style.use('ggplot')
sns.set_palette("husl")


#### OpenMeteo

In [ ]:
from weather_tools import WeatherDataFetcher

In [ ]:
fetcher = WeatherDataFetcher()

print("Variables disponibles:")
print(fetcher.get_available_variables())

In [ ]:
# Coordenadas de ejemplo (Buenos Aires)
lat = -39.422539
lon = -71.936493
start_date = "2024-12-01"
end_date = "2025-03-10"

print(f"Obteniendo datos para:")
print(f"Coordenadas: ({lat}, {lon})")
print(f"Período: {start_date} a {end_date}")

In [ ]:
df_weather = fetcher.fetch_weather_data(lat, lon, start_date, end_date)

fetcher.save_to_csv(df_weather, "datos_clima_historico.csv")

# También guardar en data/processed
processed_dir = "../data/raw/open_meteo"
os.makedirs(processed_dir, exist_ok=True)
df_weather.to_csv(f"{processed_dir}/clima_procesado.csv", index=False)

print("💾 Datos guardados exitosamente!")

In [ ]:
print("✅ Datos obtenidos exitosamente!")
print(f"📊 Dimensiones: {df_weather.shape}")
print(f"📋 Columnas: {list(df_weather.columns)}")

print("\n📅 Rango temporal:")
print(f"   Inicio: {df_weather['datetime'].min()}")
print(f"   Fin: {df_weather['datetime'].max()}")

print("\n🔍 Primeras filas:")
display(df_weather.head())

print("\n📈 Estadísticas descriptivas:")
display(df_weather.describe())

In [ ]:
if not df_weather.empty:
    # Copia + Filtro de Fecha
    df_isolated = df_weather.copy()
    df_isolated = df_isolated[df_isolated['date'] < datetime.date(2024, 12, 25)]

    fig, axes = plt.subplots(3, 2, figsize=(15, 12))

    # Temperatura
    df_isolated.plot(x='datetime', y='temperature', ax=axes[0,0], title='Temperatura (°C)', color='red')
    axes[0,0].set_ylabel('Temperatura (°C)')
    
    # Humedad
    df_isolated.plot(x='datetime', y='humidity', ax=axes[0,1], title='Humedad Relativa (%)', color='blue')
    axes[0,1].set_ylabel('Humedad (%)')
    
    # Precipitación
    df_isolated.plot(x='datetime', y='precipitation', ax=axes[1,0], title='Precipitación (mm)', color='green')
    axes[1,0].set_ylabel('Precipitación (mm)')
    
    # Velocidad del viento
    df_isolated.plot(x='datetime', y='wind_speed', ax=axes[1,1], title='Velocidad del Viento (km/h)', color='orange')
    axes[1,1].set_ylabel('Velocidad (km/h)')
    
    # Presión atmosférica
    df_isolated.plot(x='datetime', y='pressure', ax=axes[2,0], title='Presión Atmosférica (hPa)', color='purple')
    axes[2,0].set_ylabel('Presión (hPa)')
    
    # Cobertura de nubes
    df_isolated.plot(x='datetime', y='cloud_cover', ax=axes[2,1], title='Cobertura de Nubes (%)', color='gray')
    axes[2,1].set_ylabel('Cobertura (%)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
print("📊 Resumen por día:")
df_daily = df_weather.groupby('date').agg({
    'temperature': ['mean', 'min', 'max'],
    'humidity': 'mean',
    'pressure': 'mean',
    'wind_speed': 'mean',
    'precipitation': 'sum',
    'rain': 'sum',
    'snowfall': 'sum',
    'sunshine_hours': 'sum'
}).round(2)

df_daily.columns = [
    'temp_promedio', 'temp_min', 'temp_max', 
    'humedad_promedio', 'presion_promedio', 
    'viento_promedio', 'precipitacion_total',
    'lluvia_total', 'nieve_total', 'horas_sol'
]

display(df_daily)

In [ ]:
print("🔗 Matriz de correlación:")
numeric_cols = ['temperature', 'humidity', 'pressure', 'wind_speed', 'precipitation']
corr_matrix = df_weather[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Matriz de Correlación - Variables Meteorológicas')
plt.tight_layout()
plt.savefig('../docs/correlacion_clima.png', dpi=300, bbox_inches='tight')
plt.show()


#### CDSAPI

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Agregar scripts al path
sys.path.append('../scripts')

# Configuración
plt.style.use('ggplot')
sns.set_palette("husl")

In [ ]:
from cds_tools import CDSTools

In [ ]:
# Obtener datos ERA5
lat = -39.422539
lon = -71.936493
start_date = "2024-12-01"
end_date = "2024-12-01"

cds = CDSTools()

# Obtener datos de temperatura
df_temp = cds.get_climate_data('2m_temperature', start_date, end_date, lat, lon, radius = 0.5, format = 'netcdf')


#### FIRMS

In [ ]:
from firms_tools import FIRMSTools
from project_config import ProjectConfig

# Inicializar herramientas
config = ProjectConfig()
firms = FIRMSTools(config)

# Descargar datos
print("Descargando datos de FIRMS...")
# raw_data = firms.download_fire_data(source='MODIS_SP')
raw_data = firms.get_multisource_fire_data()

# if raw_data is not None and not raw_data.empty:
#     print(f"✓ Datos descargados exitosamente: {len(raw_data)} registros")
#     print(f"Columnas: {list(raw_data.columns)}")
    
#     # Procesar datos
#     processed_data = firms.process_fire_data(raw_data)
#     print(f"Datos procesados: {len(processed_data)} registros")
    
#     # Mostrar información básica
#     print(f"\n📊 Resumen de datos:")
#     print(f"Rango temporal: {processed_data['acq_date'].min()} to {processed_data['acq_date'].max()}")
#     print(f"Número de días con datos: {processed_data['acq_date'].nunique()}")
    
#     if 'confidence' in processed_data.columns:
#         print(f"Distribución de confianza:")
#         print(processed_data['confidence'].value_counts())
    
#     # Visualización rápida
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
#     # Distribución temporal
#     daily_counts = processed_data['acq_date'].value_counts().sort_index()
#     daily_counts.plot(ax=ax1, kind='bar', title='Detecciones por día')
#     ax1.tick_params(axis='x', rotation=45)
    
#     # Mapa de detecciones
#     geometry = [Point(xy) for xy in zip(processed_data.longitude, processed_data.latitude)]
#     gdf = gpd.GeoDataFrame(processed_data, geometry=geometry, crs="EPSG:4326")
    
#     # Cargar mapa base de Argentina
#     world = gpd.read_file("https://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip")
#     argentina = world[world.NAME == 'Argentina']
    
#     argentina.plot(ax=ax2, color='lightgray', edgecolor='black')
#     gdf.plot(ax=ax2, color='red', markersize=10, alpha=0.7, label='Detecciones')
#     ax2.set_title('Ubicación de detecciones de incendios')
#     ax2.legend()
    
#     plt.tight_layout()
#     plt.show()
# else:
#     print("❌ No se pudieron obtener datos. Revisa los logs para más detalles.")

In [ ]:
import logging
from spatial_analysis import SpatialAnalysis
from project_config import DATA_PROCESSED

# Configuración
logging.basicConfig(level=logging.INFO)
plt.style.use('ggplot')

# Cargar datos
print("📊 Cargando datos de incendios...")
fire_data = pd.read_csv('../data/raw/firms/firms_MODIS_SP_2024-12-20_2024-12-29.csv')

# Mostrar información básica
print(f"Total de detecciones: {len(fire_data)}")
print(f"Rango temporal: {fire_data['acq_date'].min()} to {fire_data['acq_date'].max()}")
print(f"Columnas disponibles: {list(fire_data.columns)}")

# Inicializar análisis espacial
print("\n🔍 Iniciando análisis espacial...")
spatial_analyzer = SpatialAnalysis(fire_data)

# 1. Crear GeoDataFrame
gdf = spatial_analyzer.create_geodataframe()
print(f"GeoDataFrame creado: {len(gdf)} features")

# 2. Calcular estadísticas básicas
stats = spatial_analyzer.calculate_basic_stats()
print("\n📈 Estadísticas espaciales:")
for key, value in stats.items():
    print(f"{key}: {value}")

# 3. Visualización estática
print("\n🎨 Generando visualizaciones...")
spatial_analyzer.plot_spatial_distribution(save_path = '../docs/Entregas/spatial_analysis.png')

# 4. Análisis de clusters
print("\n📊 Analizando clusters espaciales...")
cluster_stats = spatial_analyzer.cluster_analysis()
print("Estadísticas por cluster:")
print(cluster_stats)

# 5. Mapa interactivo (opcional - requiere folium)
try:
    print("\n🌍 Creando mapa interactivo...")
    interactive_map = spatial_analyzer.plot_interactive_map(save_path='../docs/Entregas/interactive_map.html')
    print("Mapa interactivo guardado como 'interactive_map.html'")
except ImportError:
    print("Folium no instalado. Instala con: pip install folium")

# 6. Exportar resultados
print("\n💾 Exportando resultados...")
spatial_analyzer.export_results(DATA_PROCESSED)
print("Análisis completado! ✅")

# Análisis adicional por día/noche
print("\n🌙 Análisis día vs noche:")
day_night_stats = gdf.groupby('daynight').agg({
    'frp': ['mean', 'sum', 'count'],
    'confidence': 'mean',
    'brightness': 'mean'
}).round(2)
print(day_night_stats)

# Análisis por satélite
print("\n🛰️ Análisis por satélite:")
satellite_stats = gdf.groupby('satellite').agg({
    'frp': ['mean', 'sum', 'count'],
    'confidence': 'mean'
}).round(2)
print(satellite_stats)